In [1]:
import pandas as pd
import numpy as np

Fields
- Postal code
- Address
- Latitute/Longtitude 
- Flat type
- Monthly Electricity Consumption (kWh)
- Electricity Provider
  

Attributes:
- Address obtained from a postal code web scrapping from onemap.sg API, 25k data point per month
- electricity_provider and flat_types randomly generated
- Data created for January 2023 to June 2024
- Consumption base randomly generated within consumption range
consumption_ranges = {
    "HDB 1-Room": (50, 150),
    "HDB 2-Room": (100, 200),
    "HDB 3-Room": (150, 300),
    "HDB 4-Room": (200, 400),
    "HDB 5-Room": (250, 500),
    "HDB Executive": (300, 600)}
- Multiplier for retailers - 1.05x for Tuas Power and Sembcorp, and 0.95x for Geneco
- Multiplier for season - 1.1x for March to October
- Electricity consumption are slowly increasing - 1% increase every month from March 2023
- 0.5% chance may sometimes contain null for electricity consumption

In [2]:
retailers = ["Geneco", "Keppel Electric", "Senoko Energy", "Tuas Power", "Pacific Light", "Sembcorp"]
flat_types = ["HDB 1-Room", "HDB 2-Room", "HDB 3-Room", "HDB 4-Room", "HDB 5-Room", "HDB Executive"]

In [3]:
postal_code_mapping = pd.read_csv("sg_zipcode_mapper.csv", encoding='ISO-8859-1').drop("postal.1", axis=1)
postal_code_mapping.head()

,postal,latitude,longtitude,searchval,blk_no,road_name,building,address
0,398614,1.312763,103.883519,# 1 LOFT,1,LORONG 24 GEYLANG,# 1 LOFT,1 LORONG 24 GEYLANG # 1 LOFT SINGAPORE 398614
1,398721,1.312390,103.881504,# 1 SUITES,1,LORONG 20 GEYLANG,# 1 SUITES,1 LORONG 20 GEYLANG # 1 SUITES SINGAPORE 398721
2,629875,1.309135,103.679463,1 BENOI ROAD SINGAPORE 629875,1,BENOI ROAD,NIL,1 BENOI ROAD SINGAPORE 629875
3,439731,1.305466,103.895674,1 BOSCOMBE ROAD SINGAPORE 439731,1,BOSCOMBE ROAD,NIL,1 BOSCOMBE ROAD SINGAPORE 439731
4,659592,1.344619,103.749789,1 BUKIT BATOK STREET 22 SINGAPORE 659592,1,BUKIT BATOK STREET 22,NIL,1 BUKIT BATOK STREET 22 SINGAPORE 659592


In [4]:
consumption_ranges = {
    # mean, stddev
    "HDB 1-Room": [2, 2],
    "HDB 2-Room": [3, 3],
    "HDB 3-Room": [4.5, 4.5],
    "HDB 4-Room": [6, 6],
    "HDB 5-Room": [7.5, 7.5],
    "HDB Executive": [9, 9]
}


def adjust_for_season(month, base_consumption):
    # Example seasonal adjustment: slightly higher consumption in the hotter months (March to October)
    if month in [3, 4, 5, 6, 7, 8, 9, 10]:
        return base_consumption * 1.2
    else:
        return base_consumption


def adjust_for_provider(electricty_provider, base_consumption):
    # Example seasonal adjustment: slightly higher consumption in the hotter months (March to October)
    if electricty_provider in ["Tuas Power", "Sembcorp"]:
        return base_consumption * 1.1
    else:
        return base_consumption

def adjust_for_day_of_week(day_of_week, base_consumption):
    if day_of_week in [5, 6]:
        base_consumption *= 1.15  # Increase consumption by 5% on weekends
    else:
        base_consumption *= 1  # Decrease consumption by 5% on weekdays
    return base_consumption


def generate_consumption(flat_type):
    # return np.random.uniform(*consumption_ranges[flat_type])
    consumption = np.random.normal(consumption_ranges[flat_type][0], consumption_ranges[flat_type][1])
    
    # Ensure the generated value falls within the specified range for the flat type
    consumption = max(consumption, 0)
    
    return consumption


In [5]:
date_range = pd.date_range(start='2023-01-01', end='2024-06-01', freq='MS')
# date_range = pd.date_range(start='2023-01-01', end='2023-02-28', freq='MS')

# fixed
df = pd.DataFrame(postal_code_mapping)
df['flat_type'] = np.random.choice(flat_types, len(df))
df['electricity_provider'] = np.random.choice(retailers, len(df))

df.drop(['latitude', 'longtitude', 'blk_no', 'road_name', 'building', 'address'], axis=1, inplace=True)

monthly_increase_multiplier = 1

for month in date_range:
    # month level
    dfs_for_month = []

    month_start = month
    month_end = month_start + pd.offsets.MonthEnd(0)\

    date_range_month = pd.date_range(start=month_start, end=month_end, freq='D')

    # day level
    for date in date_range_month:
        df_date = df.copy()
        df_date['date'] = date.strftime('%Y-%m-%d')
        df_date["month"] = month.strftime('%Y-%m-%d')
        df_date["day_of_week"] = date.weekday()        
        df_date['daily_electricity_consumption_kWh'] = df_date['flat_type'].apply(generate_consumption)
        df_date['daily_electricity_consumption_kWh'] = df_date['daily_electricity_consumption_kWh']*monthly_increase_multiplier
        df_date['daily_electricity_consumption_kWh'] = df_date.apply(lambda row: adjust_for_day_of_week(row['day_of_week'], row['daily_electricity_consumption_kWh']), axis=1)
        df_date['daily_electricity_consumption_kWh'] = df_date.apply(lambda row: adjust_for_season(row['month'], row['daily_electricity_consumption_kWh']), axis=1)
        df_date['daily_electricity_consumption_kWh'] = df_date.apply(lambda row: adjust_for_provider(row['electricity_provider'], row['daily_electricity_consumption_kWh']), axis=1)
    
        dfs_for_month.append(df_date)
    
    month_df = pd.concat(dfs_for_month, ignore_index=True)

    # # Randomly set 0.5% of monthly_electricity_consumption_kWh values to null
    num_nulls = int(0.005 * len(month_df))
    null_indices = np.random.choice(month_df.index, num_nulls, replace=False)
    month_df.loc[null_indices, 'daily_electricity_consumption_kWh'] = None
    
    monthly_increase_multiplier = monthly_increase_multiplier + 0.005

    month_df = month_df[['date', 'month', 'postal', 'flat_type', 'electricity_provider', 'daily_electricity_consumption_kWh']]
    
    # print(month, len(month_df), round(sum(month_df.daily_electricity_consumption_kWh)))

    month_df.to_csv(f"{date.year}_{str(date.month).zfill(2)}_electricity_consumption.csv", index=False)